<a href="https://colab.research.google.com/github/Aravinnthram/CDW-Intern/blob/Day5/Copy_of_GPT_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, GPT2LMHeadModel, pipeline, \
                         Trainer, TrainingArguments

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')  # load up a standard gpt2 model

tokenizer.pad_token = tokenizer.eos_token
# set our pad token to be the eos token. This lets gpt know how to fill space

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
# load up our data into a dataset
pds_data = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/Unit I Upd_0.txt',  # Principles of Data Science - Sinan Ozdemir
    block_size=64  # length of each chunk of text to use as a datapoint
)

/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
pds_data[0], pds_data[0].shape  # inspect the first point

(tensor([26453,   220,   784,   220,   220,   314,  3268,  5446, 28644,  2849,
           220,   220,   198, 39688,   290,  2106,   286, 22299, 13442, 18252,
          2267,   220,   220,   198,  3041,   259, 13442, 18252,   357,  7836,
             8,   318,   257,   850,  3245,   286,  4572,  4673,  7867,   416,
         17211,   220,   198, 23947,  1435,    11,   220, 10759,   220,   319,
           220,   703,   220,  6554,   220,   460,   220,  2193,   220,   284,
           220,   787,   220,  5370]),
 torch.Size([64]))

In [ ]:
print(tokenizer.decode(pds_data[0]))

Unit  –   I INTRODUCTION  
Origin and history of Reinforcement Learning research  
Reinforcement Learning (RL) is a subfield of machine learning inspired by behavioral 
psychology,  focusing  on  how  agents  can  learn  to  make  decisions


In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
    # MLM is Masked Language Modelling (for BERT + auto-encoding tasks)
)

In [ ]:
# example of how collator pads data dynamically
collator_example = data_collator([tokenizer('I am an input'), tokenizer('So am I')])

collator_example

{'input_ids': tensor([[   40,   716,   281,  5128],
        [ 2396,   716,   314, 50256]]), 'attention_mask': tensor([[1, 1, 1, 1],
        [1, 1, 1, 0]]), 'labels': tensor([[  40,  716,  281, 5128],
        [2396,  716,  314, -100]])}

In [ ]:
collator_example.input_ids  # 50256 is our pad token id

tensor([[   40,   716,   281,  5128],
        [ 2396,   716,   314, 50256]])

In [ ]:
tokenizer.pad_token_id

50256

In [ ]:
collator_example.attention_mask  # Note the 0 in the attention mask where we have a pad token

tensor([[1, 1, 1, 1],
        [1, 1, 1, 0]])

In [ ]:
collator_example.labels  # note the -100 to ignore loss calculation for the padded token
# Labels are shifted inside the GPT model so we don't need to worry about that

tensor([[  40,  716,  281, 5128],
        [2396,  716,  314, -100]])

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')  # load up a GPT2 model

pretrained_generator = pipeline(  # create a generator with built in params
    'text-generation', model=model, tokenizer='gpt2',
    config={'max_length': 200, 'do_sample': True, 'top_p': 0.9, 'temperature': 0.7, 'top_k': 10}
)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
print('----------')
for generated_sequence in pretrained_generator('This dataset shows the relationship', num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print('----------')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


----------
This dataset shows the relationship between distance traveled and speed on the surface of a comet at the time of it impact, the time period when a comet collided with a target and the estimated gravitational potential of the region. Here, we see there is an inverse
----------
This dataset shows the relationship between the number of patients who were classified as having any type of gastrointestinal illness before the year 2003–2005 and overall mortality statistics, mortality data on mortality at the end of the year, or number of deaths per 100 000 population
----------
This dataset shows the relationship between changes to each of the five levels and the change in average household income between the beginning of 1990 and the end of 1995 for three of the four income groups. The most interesting finding in the new dataset is the distribution of
----------


In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2_pds", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    logging_steps=10,
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=pds_data.examples[:int(len(pds_data.examples)*.8)],
    eval_dataset=pds_data.examples[int(len(pds_data.examples)*.8):]
)

trainer.evaluate()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


{'eval_loss': 4.319733619689941,
 'eval_model_preparation_time': 0.0039,
 'eval_runtime': 0.3957,
 'eval_samples_per_second': 121.306,
 'eval_steps_per_second': 5.054}

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,No log,3.778177,0.003900
2,3.741800,3.597900,0.003900
3,3.741800,3.546611,0.003900


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=18, training_loss=3.5382812288072376, metrics={'train_runtime': 47.808, 'train_samples_per_second': 11.86, 'train_steps_per_second': 0.377, 'total_flos': 18519072768000.0, 'train_loss': 3.5382812288072376, 'epoch': 3.0})

In [ ]:
trainer.evaluate()  # loss decrease is slowing down so we are hitting our limit

{'eval_loss': 3.5466108322143555,
 'eval_model_preparation_time': 0.0039,
 'eval_runtime': 0.2974,
 'eval_samples_per_second': 161.372,
 'eval_steps_per_second': 6.724,
 'epoch': 3.0}

In [ ]:
trainer.save_model()

In [ ]:
loaded_model = GPT2LMHeadModel.from_pretrained('./gpt2_pds')

finetuned_generator = pipeline(
    'text-generation', model=loaded_model, tokenizer=tokenizer,
    config={'max_length': 2000, 'do_sample': True, 'top_p': 0.9, 'temperature': 0.7, 'top_k': 10}
)

Device set to use cuda:0


In [ ]:
# examples are now sustainably about data
print('----------')
for generated_sequence in finetuned_generator('This dataset shows the relationship', num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print('----------')

----------
This dataset shows the relationship between the number of samples in a set of a variable and its corresponding score on the GSE of that variable. 
Note :  The standard deviation of a score is the sum of the number of samples in a set
----------
This dataset shows the relationship between a given variable and the corresponding probability distribution over time (P= 0.05), based on the following formula. 
(a, b: P=  + −1/0 )  
(1
----------
This dataset shows the relationship of the distribution of values between variables, and can be used to make a decision in order to make an ethical judgement: consider which parameter(s) should be considered and which value is best for the given choice: (a
----------
